# Tracing SavePageNow

In order to better understand the internal workings of SavePageNow and how they get imprinted in the collected WARC data we ran a series of experiments on on October 25, 2018 to archive a discrete set of URLs with three clients: 

* the SavePageNow form at [https://web.archive.org/](https://web.archive.org)
* the [Wayback Machine Firefox Extension](https://addons.mozilla.org/en-US/firefox/addon/wayback-machine_new/)
* a Python bot [spn-probe](https://github.com/edsu/spn-probe/)

In each case we archived a distinct URL which identified the client, and the time so that we could reliably identify the relevant WARC records later. For example:

* web form: https://mith.umd.edu/research/?ua=firefox&t=20181024230000
* browser extension: https://mith.umd.edu/research/?ua=extension&t=20181024230000
* spn-probe: https://web.archive.org/save/https://mith.umd.edu/research/?ua=spn-probe&t=20181024140003

## Locating the Records

In order to locate the records


